# Documention Agent

This notebook demonstrates a documention agent:
1. Video-to-protocol conversion using Vertex AI
2. With knowledge from few-shot-learning (video and protocol examples)

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import configparser

import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [7]:
def upload_few_shot_examples(
    few_shot_example_folder: str,
    few_shot_examples: list[str],
    bucket: object,
    subfolder_in_bucket: str,
) -> dict[str, dict[str, str]]:
    """Upload few-shot example videos and protocols to Google Cloud Storage.

    Args:
        few_shot_example_folder: Path to folder containing few-shot examples
        few_shot_examples: List of example names (without file extensions)
        bucket: GCS bucket object to upload to
        subfolder_in_bucket: Name of subfolder in bucket

    Returns:
        Dictionary mapping example names to dictionaries containing video and protocol URIs

    """
    uploaded_uris = {}

    for example in few_shot_examples:
        # Upload video file
        video_path = Path(few_shot_example_folder) / f"{example}.mp4"
        video_uri = video_to_protocol.upload_video_to_gcs(
            video_path, bucket, subfolder_in_bucket
        )

        # Upload protocol file
        protocol_path = Path(few_shot_example_folder) / f"{example}.md"
        protocol_uri = video_to_protocol.upload_video_to_gcs(
            protocol_path, bucket, subfolder_in_bucket
        )

        # Store the URIs in a dictionary
        uploaded_uris[example] = {"video": video_uri, "protocol": protocol_uri}

    return uploaded_uris

In [8]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [9]:
# Upload few shot examples to Google Cloud Storage

subfolder_in_bucket = "fewShotExamples"

few_shot_example_folder = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/"

few_shot_examples = [
    "tims_calibration",
    "Refilling_tuneMix_at_timsTOFUltra",
    "Connect_ionOpticks_column",
    "Disconnect_IonOpticks_column_from_sample_line",
    "Placing_Evotips_on_Evosep",
]

uploaded_examples = upload_few_shot_examples(
    few_shot_example_folder, few_shot_examples, bucket, subfolder_in_bucket
)
uploaded_examples

{'tims_calibration': {'video': 'gs://mannlab_videos/fewShotExamples/tims_calibration.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/tims_calibration.md'},
 'Refilling_tuneMix_at_timsTOFUltra': {'video': 'gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Refilling_tuneMix_at_timsTOFUltra.md'},
 'Connect_ionOpticks_column': {'video': 'gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Connect_ionOpticks_column.md'},
 'Disconnect_IonOpticks_column_from_sample_line': {'video': 'gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Disconnect_IonOpticks_column_from_sample_line.md'},
 'Placing_Evotips_on_Evosep': {'video': 'gs://mannlab_videos/fewShotExamples/Placing_Evotips_on_Evosep.mp4',
  'protocol': 'gs://mannlab_videos/fewShotExamples/Placing_Evotips

In [10]:
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/ready_examples/Disconnect_IonOpticks_column_from_sample_line.mp4"
video_uri_input = video_to_protocol.upload_video_to_gcs(video_path, bucket)

In [11]:
system_instructions = """
You are a highly observant research assistant in Professor Matthias Mann's proteomics and mass spectrometry laboratory. Your expertise lies in detailed documentation of experimental procedures.

Analyze the video and reconstruct a step-by-step protocol by focusing on the actions in the video. Focus on user interactions with equipment, devices, and software. The goal is a clear, concise, unambiguous protocol reproducible by someone with no prior knowledge. "Think aloud" as if you were the researcher in the video that describes their work. Describe what you see at every secound.
Your task is to analyze the provided video and extract a detailed experimental protocol.
You always stick to the facts in the sources provided, and never make up new facts.
Take a deep breath and think step by step.
Answer direct.
"""

In [12]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

inputs = []

# few shot examples
for _i, uris in enumerate(uploaded_examples.values(), 1):
    video_expl_uri = uris["video"]
    file_expl_uri = uris["protocol"]

    inputs.extend(
        [
            "Example Video {num}:",
            Part.from_uri(video_expl_uri, mime_type="video/mp4"),
            "Example Protocol {num}:",
            Part.from_uri(file_expl_uri, mime_type="text/md"),
        ]
    )

# system prompts, and processed video
inputs.extend(
    [
        system_instructions,
        "Video:",
        Part.from_uri(video_uri_input, mime_type="video/mp4"),
        "Protocol:",
    ]
)

model = GenerativeModel("gemini-1.5-flash-002")
response = model.generate_content(inputs, generation_config={"temperature": 0})
observation = response.text
print(response.usage_metadata)
Markdown(observation)

prompt_token_count: 134355
candidates_token_count: 498
total_token_count: 134853
prompt_tokens_details {
  modality: VIDEO
  token_count: 129675
}
prompt_tokens_details {
  modality: TEXT
  token_count: 4680
}
candidates_tokens_details {
  modality: TEXT
  token_count: 498
}



Here is a step-by-step protocol based on the provided video.  I will describe what is happening in each section of the video.

**Protocol: Connecting Evotips to the Evosep One System**

**Abstract:** This protocol details the procedure for preparing Evotips and loading them into the Evosep One system.

**Materials:**

* Evosep One System
* Evotips (Evosep, 100 x 96 tips, cat. no. EV2018)
* Evotip boxes (96-well format)
* Buffer A (0.1% (vol/vol) FA)

**Procedure:**

1. **Inspect Evotips:** Examine each Evotip to ensure it meets quality standards.  A properly prepared Evotip will have a pale-colored SPE material disc with visible solvent above it. Discard any Evotips that are dry or have a white disc. (0:00-0:14)

2. **Prepare Evotip Boxes:** Place the Evotip boxes into the Evosep rack system.  Ensure each box is firmly seated. Fill each box with Buffer A solution to a depth of at least 1 cm. (0:14-0:16)

3. **Load Evotips:** Carefully place the inspected Evotips into the prepared Evotip boxes.  (0:16-0:20)

4. **Document Tip Positions:** Record the location of each Evotip using the Evosep's standardized positioning system.  The rack has two columns (left and right). The left column (top to bottom) is labeled S1, S2, S3. The right column (top to bottom) is labeled S4, S5, S6. Each box uses the standard 96-well format (A1-F12). (0:20-0:21)


**Expected Results:**

* Evotip boxes are securely positioned in the rack system.
* Buffer A solution is present in each box (minimum 1 cm depth).
* All Evotips have pale-colored SPE material discs and a visible solvent meniscus.
* Tip positions are accurately documented.

**Note:** The video shows the researcher using three Evotips, placing them in positions A1-A3 of a single Evotip box.  The protocol is generalized to accommodate multiple boxes and positions.